In [1]:
import pandas as pd
import math
from collections import Counter

df_tennis = pd.read_csv('data4.csv')
print("\n given Play Tennis Data set:\n \n",df_tennis)

def entropy(probs):
    return sum([-prob*math.log(prob,2) for prob in probs])

def entropy_of_list(a_list):
    cnt = Counter(x for x in a_list)
    num = len(a_list)*1.0
    probs = [x/num for x in cnt.values()]
    return entropy(probs)

def information_gain(df, split_attribute_name,target_attribute_name):
    df_split = df.groupby(split_attribute_name)
    nobs = len(df.index)*1.0
    df_aggent = df_split.agg({target_attribute_name:[entropy_of_list, lambda x:len(x)/nobs]})[target_attribute_name]
    df_aggent.columns = ['Entropy','PropObservation']
    new_entropy = sum(df_aggent['Entropy']*df_aggent['PropObservation'])
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy-new_entropy

def id3(df,target_attribute_name,attribute_names,default_class=None):
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt) == 1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class  
    else:
        default_class = max(cnt.keys())
        gainz = [information_gain(df,attr,target_attribute_name) for attr in attribute_names]
        print("Gain=",gainz)
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_names[index_of_max]
        print("Best Attribute",best_attr)
        tree = {best_attr:{}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        for attr,data in df.groupby(best_attr):
            print(attr)
            print(data)
            subtree = id3(data,target_attribute_name,remaining_attribute_names,default_class)
            tree[best_attr][attr] = subtree
        return tree
            
attribute_names = list(df_tennis.columns)
print("List of Atrributes:", attribute_names)    
attribute_names.remove('PlayTennis')
print('Predicting Attributes:',attribute_names)

from pprint import pprint
tree = id3(df_tennis,'PlayTennis',attribute_names)
print("\n\nThe Resultant decision tree:\n")
pprint(tree)


 given Play Tennis Data set:
 
    PlayTennis   Outlook Temperature Humidity    Wind
0          No     Sunny         Hot     High    Weak
1          No     Sunny         Hot     High  Strong
2         Yes  Overcast         Hot     High    Weak
3         Yes      Rain        Mild     High    Weak
4         Yes      Rain        Cool   Normal    Weak
5          No      Rain        Cool   Normal  Strong
6         Yes  Overcast        Cool   Normal  Strong
7          No     Sunny        Mild     High    Weak
8         Yes     Sunny        Cool   Normal    Weak
9         Yes      Rain        Mild   Normal    Weak
10        Yes     Sunny        Mild   Normal  Strong
11        Yes  Overcast        Mild     High  Strong
12        Yes  Overcast         Hot   Normal    Weak
13         No      Rain        Mild     High  Strong
List of Atrributes: ['PlayTennis', 'Outlook', 'Temperature', 'Humidity', 'Wind']
Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']
Gain= [0.246749819774